In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
import math
import random
import csv

%matplotlib inline 

# NCAA Bracket

In [2]:
# folder = 'DataFiles2018/Stage2UpdatedDataFiles'
folder = 'Stage2DataFiles'
prediction_year = 2019

In [3]:

# df_seeds = pd.read_csv(folder + '/NCAATourneySeeds.csv')
# df_slots = pd.read_csv(folder + '/NCAATourneySlots.csv')
# df_tour = pd.read_csv(folder + '/NCAATourneyCompactResults.csv')
# df_teams = pd.read_csv(folder + '/Teams.csv')
# df_massey = pd.read_csv(folder + '/MasseyOrdinals_thruSeason2018_Day128.csv')
# df_tour_detailed = pd.read_csv(folder + '/NCAATourneyDetailedResults.csv')
# df_seasons = pd.read_csv(folder + '/Stage2UpdatedDataFiles/Seasons.csv')
# df_538 = pd.read_csv(folder + '/fivethirtyeight_ncaa_forecasts.csv')
# df_reg = pd.read_csv(folder + '/RegularSeasonCompactResults.csv')
# df_reg_det = pd.read_csv(folder + '/RegularSeasonDetailedResults.csv')
# df_sample_stage2 = pd.read_csv(folder + '/SampleSubmissionStage2.csv')


## Make ELO rating from scratch

In [4]:
def calc_elo(win_team, lose_team, season):
    winner_rank = get_elo(season, win_team)
    loser_rank = get_elo(season, lose_team)

    """
    This is originally from from:
    http://zurb.com/forrst/posts/An_Elo_Rating_function_in_Python_written_for_foo-hQl
    """
    rank_diff = winner_rank - loser_rank
    exp = (rank_diff * -1) / 400
    odds = 1 / (1 + math.pow(10, exp))
    if winner_rank < 2100:
        k = 32
    elif winner_rank >= 2100 and winner_rank < 2400:
        k = 24
    else:
        k = 16
    new_winner_rank = round(winner_rank + (k * (1 - odds)))
    new_rank_diff = new_winner_rank - winner_rank
    new_loser_rank = loser_rank - new_rank_diff

    return new_winner_rank, new_loser_rank

In [5]:
base_elo = 1600
team_elos = {}  # Reset each year.
team_stats = {}

def initialize_data():
    for i in range(2003, prediction_year+1):
        team_elos[i] = {}
        team_stats[i] = {}

In [6]:
def get_elo(season, team):
    try:
        return team_elos[season][team]
    except:
        try:
            # Get the previous season's ending value.
            team_elos[season][team] = team_elos[season-1][team]
            return team_elos[season][team]
        except:
            # Get the starter elo.
            team_elos[season][team] = base_elo
            return team_elos[season][team]

In [7]:
def predict_winner(team_1, team_2, model, season, stat_fields):
    features = []

    # Team 1
    features.append(get_elo(season, team_1))
    for stat in stat_fields:
        features.append(get_stat(season, team_1, stat))

    # Team 2
    features.append(get_elo(season, team_2))
    for stat in stat_fields:
        features.append(get_stat(season, team_2, stat))

    return model.predict_proba([features])

In [8]:
def update_stats(season, team, fields):
    """
    This accepts some stats for a team and udpates the averages.
    First, we check if the team is in the dict yet. If it's not, we add it.
    Then, we try to check if the key has more than 5 values in it.
        If it does, we remove the first one
        Either way, we append the new one.
    If we can't check, then it doesn't exist, so we just add this.
    Later, we'll get the average of these items.
    """
    if team not in team_stats[season]:
        team_stats[season][team] = {}

    for key, value in fields.items():
        # Make sure we have the field.
        if key not in team_stats[season][team]:
            team_stats[season][team][key] = []

        if len(team_stats[season][team][key]) >= 15:
            team_stats[season][team][key].pop(0)
        team_stats[season][team][key].append(value)

In [9]:
def get_stat(season, team, field):
    try:
        l = team_stats[season][team][field]
        return sum(l) / float(len(l))
    except:
        return 0

In [10]:
def build_team_dict():
    team_ids = pd.read_csv(folder + '/Teams.csv')
    team_id_map = {}
    for index, row in team_ids.iterrows():
        team_id_map[row['TeamID']] = row['TeamName']
    return team_id_map

In [12]:
# Set up data for the X and y
# New way 2019

X = []
y = []

def build_season_data(all_data):
    # Calculate the elo for every game for every team, each season.
    # Store the elo per season so we can retrieve their end elo
    # later in order to predict the tournaments without having to
    # inject the prediction into this loop.
    print("Building season data.")
    for index, row in all_data.iterrows():
        # Used to skip matchups where we don't have usable stats yet.
        skip = 0

        # Get starter or previous elos.
        team_1_elo = get_elo(row['Season'], row['WTeamID'])
        team_2_elo = get_elo(row['Season'], row['LTeamID'])

        # Add 100 to the home team (# taken from Nate Silver analysis.)
        if row['WLoc'] == 'H':
            team_1_elo += 100
        elif row['WLoc'] == 'A':
            team_2_elo += 100

        # We'll create some arrays to use later.
        team_1_features = [team_1_elo]
        team_2_features = [team_2_elo]

        # Build arrays out of the stats we're tracking..
        for field in stat_fields:
            team_1_stat = get_stat(row['Season'], row['WTeamID'], field)
            team_2_stat = get_stat(row['Season'], row['LTeamID'], field)
            if team_1_stat is not 0 and team_2_stat is not 0:
                team_1_features.append(team_1_stat)
                team_2_features.append(team_2_stat)
            else:
                skip = 1

        if skip == 0:  # Make sure we have stats.
            # Randomly select left and right and 0 or 1 so we can train
            # for multiple classes.
            if random.random() > 0.5:
                X.append(team_1_features + team_2_features)
                y.append(0)
            else:
                X.append(team_2_features + team_1_features)
                y.append(1)

        # AFTER we add the current stuff to the prediction, update for
        # next time. Order here is key so we don't fit on data from the
        # same game we're trying to predict.
        if row['WFTA'] != 0 and row['LFTA'] != 0:
            stat_1_fields = {
                'Score': row['WScore'],
                'FGP': row['WFGM'] / row['WFGA'] * 100,
                'FGA': row['WFGA'],
                'FGA3': row['WFGA3'],
                '3PP': row['WFGM3'] / row['WFGA3'] * 100,
                'FTP': row['WFTM'] / row['WFTA'] * 100,
                'OR': row['WOR'],
                'DR': row['WDR'],
                'AST': row['WAst'],
                'TO': row['WTO'],
                'STL': row['WStl'],
                'BLK': row['WBlk'],
                'PF': row['WPF'],
                'POSS': row['WFGA'] + 0.475 * row['WFTA'] - row['WOR'] + row['WTO'],
                'PPP': row['WScore']/(row['WFGA'] + 0.475 * row['WFTA'] - row['WOR'] + row['WTO']),
                'EM': (row['WScore']/(row['WFGA'] + 0.475 * row['WFTA'] - row['WOR'] + row['WTO']) - 
                       row['LScore']/(row['LFGA'] + 0.475 * row['LFTA'] - row['LOR'] + row['LTO'])),
                'OffRtg': row['WOffRtg'],
                'DefRtg': row['WDefRtg'],
                'NetRtg': row['WNetRtg'],
                'AstR': row['WAstR'],
                'TOR': row['WTOR'],
                'TSP': row['WTSP'],
                'eFGP': row['WeFGP'],
                'FTAR': row['WFTAR'],
                'ORP': row['WORP'],
                'DRP': row['WDRP'],
                'RP': row['WRP']
       }
            stat_2_fields = {
                'Score': row['LScore'],
                'FGP': row['LFGM'] / row['LFGA'] * 100,
                'FGA': row['LFGA'],
                'FGA3': row['LFGA3'],
                '3PP': row['LFGM3'] / row['LFGA3'] * 100,
                'FTP': row['LFTM'] / row['LFTA'] * 100,
                'OR': row['LOR'],
                'DR': row['LDR'],
                'AST': row['LAst'],
                'TO': row['LTO'],
                'STL': row['LStl'],
                'BLK': row['LBlk'],
                'PF': row['LPF'],
                'POSS': row['LFGA'] + 0.475 * row['LFTA'] - row['LOR'] + row['LTO'],
                'PPP': row['LScore']/(row['LFGA'] + 0.475 * row['LFTA'] - row['LOR'] + row['LTO']),
                'EM': (row['LScore']/(row['LFGA'] + 0.475 * row['LFTA'] - row['LOR'] + row['LTO']) - 
                       row['WScore']/(row['WFGA'] + 0.475 * row['WFTA'] - row['WOR'] + row['WTO'])),
                'OffRtg': row['LOffRtg'],
                'DefRtg': row['LDefRtg'],
                'NetRtg': row['LNetRtg'],
                'AstR': row['LAstR'],
                'TOR': row['LTOR'],
                'TSP': row['LTSP'],
                'eFGP': row['LeFGP'],
                'FTAR': row['LFTAR'],
                'ORP': row['LORP'],
                'DRP': row['LDRP'],
                'RP': row['LRP']
            }
            update_stats(row['Season'], row['WTeamID'], stat_1_fields)
            update_stats(row['Season'], row['LTeamID'], stat_2_fields)

        # Now that we've added them, calc the new elo.
        new_winner_rank, new_loser_rank = calc_elo(
            row['WTeamID'], row['LTeamID'], row['Season'])
        team_elos[row['Season']][row['WTeamID']] = new_winner_rank
        team_elos[row['Season']][row['LTeamID']] = new_loser_rank

    return X, y




In [14]:
pd.DataFrame(team_elos)

""


In [15]:
X

[]

## Start of Final Function

In [16]:
stat_fields = ['Score','FGP', 'FGA', 'FGA3', '3PP', 'FTP', 'OR', 'DR', 'AST',
               'TO', 'STL', 'BLK', 'PF', 'POSS','PPP', 'EM', 'OffRtg','DefRtg',
               'NetRtg','AstR','TOR','TSP','eFGP','FTAR', 'ORP','DRP', 'RP']
initialize_data()
df_teams = pd.read_csv(folder + '/Teams.csv')
season_data = pd.read_csv(folder + '/RegularSeasonDetailedResults.csv')
tourney_data = pd.read_csv(folder + '/NCAATourneyDetailedResults.csv')
frames = [season_data, tourney_data]
all_data = pd.concat(frames)

In [17]:
all_data.shape

(88552, 34)

In [18]:
all_data.head()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [19]:
df_teams_win = df_teams[['TeamID', 'TeamName']].rename(columns={'TeamID':'WTeamID', 'TeamName':'WTeam'})
df_teams_loss = df_teams[['TeamID', 'TeamName']].rename(columns={'TeamID':'LTeamID', 'TeamName':'LTeam'})

df_merge = pd.merge(left=all_data, right=df_teams_win, how='left', on='WTeamID')
all_data = pd.merge(left=df_merge, right=df_teams_loss, how='left', on='LTeamID')

In [20]:

all_data.shape

(88552, 36)

In [21]:
### New Advanced metrics to add


#Points Winning/Losing Team
all_data['WPts'] = all_data.apply(lambda row: 2*row.WFGM + row.WFGM3 + row.WFTM, axis=1)
all_data['LPts'] = all_data.apply(lambda row: 2*row.LFGM + row.LFGM3 + row.LFTM, axis=1)

#Calculate Winning/losing Team Possesion Feature
wPos = all_data.apply(lambda row: 0.96*(row.WFGA + row.WTO + 0.44*row.WFTA - row.WOR), axis=1)
lPos = all_data.apply(lambda row: 0.96*(row.LFGA + row.LTO + 0.44*row.LFTA - row.LOR), axis=1)
#two teams use almost the same number of possessions in a game
#(plus/minus one or two - depending on how quarters end)
#so let's just take the average
all_data['Pos'] = (wPos+lPos)/2

In [22]:
#Offensive efficiency (OffRtg) = 100 x (Points / Possessions)
all_data['WOffRtg'] = all_data.apply(lambda row: 100 * (row.WPts / row.Pos), axis=1)
all_data['LOffRtg'] = all_data.apply(lambda row: 100 * (row.LPts / row.Pos), axis=1)
#Defensive efficiency (DefRtg) = 100 x (Opponent points / Opponent possessions)
all_data['WDefRtg'] = all_data.LOffRtg
all_data['LDefRtg'] = all_data.WOffRtg
#Net Rating = Off.Rtg - Def.Rtg
all_data['WNetRtg'] = all_data.apply(lambda row:(row.WOffRtg - row.WDefRtg), axis=1)
all_data['LNetRtg'] = all_data.apply(lambda row:(row.LOffRtg - row.LDefRtg), axis=1)
                         
#Assist Ratio : Percentage of team possessions that end in assists
all_data['WAstR'] = all_data.apply(lambda row: 100 * row.WAst / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
all_data['LAstR'] = all_data.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
#Turnover Ratio: Number of turnovers of a team per 100 possessions used.
#(TO * 100) / (FGA + (FTA * 0.44) + AST + TO)
all_data['WTOR'] = all_data.apply(lambda row: 100 * row.WTO / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
all_data['LTOR'] = all_data.apply(lambda row: 100 * row.LTO / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
                    
#The Shooting Percentage : Measure of Shooting Efficiency (FGA/FGA3, FTA)
all_data['WTSP'] = all_data.apply(lambda row: 100 * row.WPts / (2 * (row.WFGA + 0.44 * row.WFTA)), axis=1)
all_data['LTSP'] = all_data.apply(lambda row: 100 * row.LPts / (2 * (row.LFGA + 0.44 * row.LFTA)), axis=1)
#eFG% : Effective Field Goal Percentage adjusting for the fact that 3pt shots are more valuable 
all_data['WeFGP'] = all_data.apply(lambda row:(row.WFGM + 0.5 * row.WFGM3) / row.WFGA, axis=1)      
all_data['LeFGP'] = all_data.apply(lambda row:(row.LFGM + 0.5 * row.LFGM3) / row.LFGA, axis=1)   
#FTA Rate : How good a team is at drawing fouls.
all_data['WFTAR'] = all_data.apply(lambda row: row.WFTA / row.WFGA, axis=1)
all_data['LFTAR'] = all_data.apply(lambda row: row.LFTA / row.LFGA, axis=1)
                         
#OREB% : Percentage of team offensive rebounds
all_data['WORP'] = all_data.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
all_data['LORP'] = all_data.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
#DREB% : Percentage of team defensive rebounds
all_data['WDRP'] = all_data.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
all_data['LDRP'] = all_data.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)                                      
#REB% : Percentage of team total rebounds
all_data['WRP'] = all_data.apply(lambda row: (row.WDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)
all_data['LRP'] = all_data.apply(lambda row: (row.LDR + row.LOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1) 

In [23]:
all_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WeFGP,LeFGP,WFTAR,LFTAR,WORP,LORP,WDRP,LDRP,WRP,LRP
0,2003,10,1104,68,1328,62,N,0,27,58,...,0.491379,0.433962,0.310345,0.415094,0.388889,0.294118,0.705882,0.611111,0.542857,0.457143
1,2003,10,1272,70,1393,63,N,0,26,62,...,0.483871,0.402985,0.306452,0.298507,0.375000,0.416667,0.583333,0.625000,0.488636,0.511364
2,2003,11,1266,73,1437,61,N,0,24,58,...,0.482759,0.321918,0.500000,0.315068,0.435897,0.543860,0.456140,0.564103,0.447917,0.552083
3,2003,11,1296,56,1457,50,N,0,18,38,...,0.513158,0.428571,0.815789,0.306122,0.230769,0.472222,0.527778,0.769231,0.403226,0.596774
4,2003,11,1400,77,1208,71,N,0,30,61,...,0.540984,0.435484,0.213115,0.435484,0.531250,0.488372,0.511628,0.468750,0.520000,0.480000


In [24]:
X, y = build_season_data(all_data)

Building season data.


In [25]:
# Create Dataframe to make it easier to see what we have
df_X = pd.DataFrame(X, columns=['ELO', 'Score','FGP', 'FGA', 'FGA3', '3PP', 'FTP', 'OR', 'DR', 'AST',
               'TO', 'STL', 'BLK', 'PF', 'POSS','PPP', 'EM', 'OffRtg','DefRtg',
               'NetRtg','AstR','TOR','TSP','eFGP','FTAR', 'ORP','DRP', 'RP',
                'LELO', 'LScore','LFGP', 'LFGA', 'LFGA3', 'L3PP', 'LFTP', 'LOR', 'LDR', 'LAST',
               'LTO', 'LSTL', 'LBLK', 'LPF', 'LPOSS', 'LPPP', 'LEM', 'LOffRtg','LDefRtg',
               'LNetRtg','LAstR','LTOR','LTSP','LeFGP','LFTAR', 'LORP','LDRP', 'LRP'])

In [26]:
#    Original Code
#     # Fit the model.
#     print("Fitting on %d samples." % len(X))

#     model = linear_model.LogisticRegression()

#     # Check accuracy.
#     print("Doing cross-validation.")
#     print(cross_validation.cross_val_score(
#         model, numpy.array(X), numpy.array(y), cv=10, scoring='accuracy', n_jobs=-1
#     ).mean())

#     model.fit(X, y)

In [27]:
# # Fit the model.
# print("Fitting on %d samples." % len(X))


# # X_features = ['ELO','AST','PPP','3PP','EM','LELO','LAST','LPPP','L3PP','LEM']
# # Xdf = df_X.values

# # ss = StandardScaler()
# # Xs = ss.fit_transform(X)

# # logreg = LogisticRegression()
# # params = {'C': np.logspace(start=-5, stop=3, num=9)}

gb = GradientBoostingClassifier(n_estimators=250)
params = {'max_depth': [2]}
model = GridSearchCV(gb, params, scoring='neg_log_loss', refit=True)
model.fit(X, y)
print('Best log_loss: {:.4}, with best params: {}'.format(model.best_score_, model.best_params_))

# # # Check accuracy.
# # print("Doing cross-validation.")
# # print(cross_val_score(
# #     model, np.array(Xs), np.array(y), cv=10, scoring='neg_log_loss', n_jobs=-1
# # ).mean())

# # model.fit(X, y)


Best log_loss: -0.5315, with best params: {'max_depth': 2}


I first used a basic gradient boost model to see which features were the most important. Then I used a gridsearch for the actual model

In [28]:
# # Show feature importance
# model = GradientBoostingClassifier(n_estimators=250,max_depth=2)
# model.fit(X, y)

# def showFeatureImportance(my_categories):
#     fx_imp = pd.Series(model.feature_importances_, index=my_categories)
#     fx_imp /= fx_imp.max()
#     fx_imp = fx_imp.sort_values()
#     plt.figure(figsize=(20,10))
#     fx_imp.plot(kind='barh')
#     plt.savefig('Feature_Importances.png')
    
    
# showFeatureImportance(df_X.columns)


In [29]:
# Now predict tournament matchups.
print("Getting teams.")
df_teams_seeds = df_teams[['TeamID', 'TeamName']].rename(columns={'TeamName':'Team'})
seeds = pd.read_csv(folder + '/NCAATourneySeeds.csv')
seeds = pd.merge(left=seeds, right=df_teams_seeds, how='left', on='TeamID')
# for i in range(2017, 2018):
tourney_teams = []
for index, row in seeds.iterrows():
    if row['Season'] == prediction_year:
        tourney_teams.append(row['TeamID'])

Getting teams.


In [30]:
# Build our prediction of every matchup.
submission_data = []
print("Predicting matchups.")
tourney_teams.sort()
for team_a in tourney_teams:
    for team_b in tourney_teams:
        if team_a < team_b:
            prediction = predict_winner(
                team_a, team_b, model, prediction_year, stat_fields)
            label = str(prediction_year) + '_' + str(team_a) + '_' + \
                str(team_b)
            submission_data.append([label, prediction[0][0]])

Predicting matchups.


In [31]:
submission_data[:5]

[['2019_1101_1113', 0.24804091346657797],
 ['2019_1101_1120', 0.16109031162832232],
 ['2019_1101_1124', 0.2376811089230182],
 ['2019_1101_1125', 0.21698278660474346],
 ['2019_1101_1133', 0.4811107049696538]]

In [32]:
# Write the results.
print("Writing %d results." % len(submission_data))
with open(folder + '/submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['ID', 'Pred'])
    writer.writerows(submission_data)


Writing 2278 results.


In [33]:
# Now so that we can use this to fill out a bracket, create a readable
# version.
print("Outputting readable results.")
team_id_map = build_team_dict()
readable = []
less_readable = []  # A version that's easy to look up.
for pred in submission_data:
    parts = pred[0].split('_')
    less_readable.append(
        [team_id_map[int(parts[1])], team_id_map[int(parts[2])], pred[1]])
    # Order them properly.
    if pred[1] > 0.5:
        winning = int(parts[1])
        losing = int(parts[2])
        proba = pred[1]
    else:
        winning = int(parts[2])
        losing = int(parts[1])
        proba = 1 - pred[1]
    readable.append(
        [
            '%s beats %s: %f' %
            (team_id_map[winning], team_id_map[losing], proba)
        ]
    )
with open(folder + '/readable-predictions.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(readable)
with open(folder + '/less-readable-predictions.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(less_readable)

Outputting readable results.


In [39]:
# pd.read_csv('DataFiles2019/Stage2UpdatedDataFiles/submission.csv').head()
pd.read_csv(folder + '/submission.csv').head()

,ID,Pred
0,NaN,NaN
1,2019_1101_1113,0.248041
2,NaN,NaN
3,2019_1101_1120,0.161090
4,NaN,NaN


In [35]:
# df_predictions = pd.read_csv('DataFiles2019/Stage2UpdatedDataFiles/less-readable-predictions.csv',header=None)
df_predictions = pd.read_csv(folder + '/less-readable-predictions.csv',header=None)

In [36]:
pd.set_option('display.max_rows', 3000)

In [37]:
df_predictions

,0,1,2
0,Abilene Chr,Arizona St,0.248041
1,Abilene Chr,Auburn,0.161090
2,Abilene Chr,Baylor,0.237681
3,Abilene Chr,Belmont,0.216983
4,Abilene Chr,Bradley,0.481111
5,Abilene Chr,Buffalo,0.175328
6,Abilene Chr,Cincinnati,0.171373
7,Abilene Chr,Colgate,0.398199
8,Abilene Chr,Duke,0.086396
9,Abilene Chr,F Dickinson,0.582837


In [38]:
df_teams

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2019
1,1102,Air Force,1985,2019
2,1103,Akron,1985,2019
3,1104,Alabama,1985,2019
4,1105,Alabama A&M,2000,2019
5,1106,Alabama St,1985,2019
6,1107,Albany NY,2000,2019
7,1108,Alcorn St,1985,2019
8,1109,Alliant Intl,1985,1991
9,1110,American Univ,1985,2019
